In [6]:
from transformers import CLIPModel, CLIPProcessor, CLIPConfig, CLIPTextConfig, CLIPVisionConfig
from transformers import pipeline
from PIL import Image
from typing import Type
import transformers
import requests
import sys
import os 

In [5]:
CLIPTextConfig()

CLIPTextConfig {
  "attention_dropout": 0.0,
  "bos_token_id": 49406,
  "eos_token_id": 49407,
  "hidden_act": "quick_gelu",
  "hidden_size": 512,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 77,
  "model_type": "clip_text_model",
  "num_attention_heads": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "projection_dim": 512,
  "transformers_version": "4.41.2",
  "vocab_size": 49408
}

In [17]:
clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [2]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [8]:
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

In [11]:
outputs = clip(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

In [4]:
from transformers import AutoImageProcessor, AutoTokenizer, AutoBackbone, AutoProcessor, AutoModel, AutoFeatureExtractor, CLIPVisionConfig, CLIPTokenizer, CLIPTokenizerFast
from transformers import PreTrainedTokenizerFast, PreTrainedTokenizer, CLIPImageProcessor

/home/antony/Documents/Visual Studio Code/clip/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
clip:CLIPModel = CLIPModel(CLIPConfig())
tok:CLIPTokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
tok_fast:CLIPTokenizerFast = CLIPTokenizerFast.from_pretrained("openai/clip-vit-base-patch32")

In [5]:
vis = CLIPImageProcessor()
img = Image.open("./image.jpg")

In [15]:
a = {"do_resize" : False}
v = CLIPImageProcessor(**a)
v

CLIPImageProcessor {
  "_valid_processor_keys": [
    "images",
    "do_resize",
    "size",
    "resample",
    "do_center_crop",
    "crop_size",
    "do_rescale",
    "rescale_factor",
    "do_normalize",
    "image_mean",
    "image_std",
    "do_convert_rgb",
    "return_tensors",
    "data_format",
    "input_data_format"
  ],
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": false,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "CLIPImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [18]:
CLIPImageProcessor().do_resize

True

In [6]:
text = ["a photo of a cat", "a photo of a dog"]
tex_tok = tok(text=text, return_tensors="pt")
tex_tok_fast = tok_fast(text=text, return_tensors="pt")
print(tex_tok)
tex_tok_fast

{'input_ids': tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}


{'input_ids': tensor([[49406,   320,  1125,   539,   320,  2368, 49407],
        [49406,   320,  1125,   539,   320,  1929, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [27]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [29]:
inputs = processor(text=text, images=img, return_tensors="pt")

In [32]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])

In [7]:
vis_img = vis(img, return_tensors="pt")

In [59]:
vis_img['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [66]:
print(tex_tok['input_ids'].shape)
tex_tok['attention_mask'].shape

torch.Size([2, 7])


torch.Size([2, 7])

In [62]:
print(inputs['input_ids'].shape)
print(inputs['attention_mask'].shape)
inputs['pixel_values'].shape

torch.Size([2, 7])
torch.Size([2, 7])


torch.Size([1, 3, 224, 224])

In [68]:
config = open("./conf/clip", 'r')
model = eval()
img_proc = eval()
token = eval()
vis_conf = eval()
model_conf = eval()


SyntaxError: invalid syntax (2240957684.py, line 1)

In [10]:
outputs = clip(**tex_tok, **vis_img)
outputs

CLIPOutput(loss=None, logits_per_image=tensor([[-0.0287,  0.0963]], grad_fn=<TBackward0>), logits_per_text=tensor([[-0.0287],
        [ 0.0963]], grad_fn=<MulBackward0>), text_embeds=tensor([[ 0.0033, -0.0239, -0.0418,  ...,  0.0089,  0.0130,  0.0215],
        [-0.0026, -0.0294, -0.0470,  ...,  0.0107,  0.0044,  0.0336]],
       grad_fn=<DivBackward0>), image_embeds=tensor([[ 9.2839e-02,  1.1504e-02, -7.5261e-03, -1.6096e-04,  4.7651e-02,
          4.3149e-02,  4.8015e-03, -4.9286e-02, -1.9040e-02, -9.5711e-03,
          1.7520e-03,  5.8258e-03, -1.9702e-02, -6.5934e-02,  1.1361e-01,
          6.2154e-02,  1.3514e-02, -4.0542e-03,  2.2011e-02, -2.6067e-02,
          3.6940e-02,  5.8862e-02,  8.4111e-03, -5.5755e-03,  3.4440e-02,
         -3.0807e-02,  7.8435e-03, -5.1666e-02,  3.9675e-02, -4.9070e-02,
          2.0484e-02,  6.2221e-02,  4.4328e-02, -2.5514e-02,  2.1468e-02,
          6.0735e-02, -1.5880e-02, -4.5636e-02,  3.5849e-02,  1.2744e-02,
         -4.3370e-02,  4.8089e-02,  4.4

In [12]:
outputs.keys()

odict_keys(['logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])

In [13]:
outputs['text_model_output']

BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.2479, -0.6767, -1.3443,  ...,  1.2699, -0.6795,  0.3467],
         [-0.0045, -0.0625, -1.3425,  ...,  0.9914,  0.0026,  0.8557],
         [ 0.2571, -0.1253, -1.3175,  ...,  1.1783, -0.0565,  0.9860],
         ...,
         [-0.1590, -0.5805, -1.0227,  ...,  1.1058, -0.1873,  2.2285],
         [ 0.0784, -0.5903, -1.1293,  ...,  0.9678, -0.3156,  2.4961],
         [ 0.2900, -1.0100, -1.3573,  ...,  0.5628, -0.5629,  2.5505]],

        [[ 0.2479, -0.6767, -1.3443,  ...,  1.2699, -0.6795,  0.3467],
         [-0.0045, -0.0625, -1.3425,  ...,  0.9914,  0.0026,  0.8557],
         [ 0.2571, -0.1253, -1.3175,  ...,  1.1783, -0.0565,  0.9860],
         ...,
         [-0.1590, -0.5805, -1.0227,  ...,  1.1058, -0.1873,  2.2285],
         [-0.2508, -0.5315, -0.9948,  ...,  0.9689, -0.5137,  2.3148],
         [-0.0165, -0.8730, -1.2669,  ...,  0.7804, -0.7680,  2.3211]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.2

In [5]:
cfg = CLIPTextConfig.from_pretrained("openai/clip-vit-base-patch32")

/home/antony/Documents/Visual Studio Code/clip/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
CLIPModel.config_class()

CLIPConfig {
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 512,
  "text_config": {
    "model_type": "clip_text_model"
  },
  "transformers_version": "4.41.2",
  "vision_config": {
    "model_type": "clip_vision_model"
  }
}

In [52]:
a = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [59]:
CLIPTextConfig()

CLIPTextConfig {
  "attention_dropout": 0.0,
  "bos_token_id": 49406,
  "eos_token_id": 49407,
  "hidden_act": "quick_gelu",
  "hidden_size": 512,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 77,
  "model_type": "clip_text_model",
  "num_attention_heads": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "projection_dim": 512,
  "transformers_version": "4.41.2",
  "vocab_size": 49408
}

In [1]:
from clip import PRE_TRAINED

/home/antony/Documents/Visual Studio Code/clip/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:

PRE_TRAINED.items()

"dict_items([('VIT_BASE_PATCH16', 'openai/clip-vit-base-16'), ('VIT_BASE_PATCH32', 'openai/clip-vit-base-32'), ('VIT_LARGE_PATCH14', 'openai/clip-vit-large-patch14'), ('VIT_LARGE_PATCH14_IMGSIZE336', 'openai/clip-vit-large-patch14-336'), ('QUALCOMM', 'qualcomm/OpenAI-Clip')])"